# PLDM TwoRooms (L1 vs L2)

Upload the tarball created by `PLDM_hieral/colab_pack.sh`, or clone the repo manually.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
OUTPUT_ROOT = '/content/drive/MyDrive/PLDM_2mvs6m'
os.makedirs(OUTPUT_ROOT, exist_ok=True)


Mounted at /content/drive


In [ ]:
from google.colab import files
import os

uploaded = files.upload()
if uploaded:
    tar_name = next(iter(uploaded))
    os.system(f"tar -xzf {tar_name}")
else:
    print("No tarball uploaded. Clone the repo manually if needed.")

repo_root = "."
if os.path.isdir("PLDM_hieral"):
    repo_root = "."
elif os.path.isdir("PLDM/PLDM_hieral"):
    repo_root = "PLDM"
if repo_root != ".":
    os.chdir(repo_root)
print("Repo root:", os.getcwd())

# Optional:
# !git clone <repo-url>
# %cd PLDM

Saving pldm_colab.tar.gz to pldm_colab.tar.gz
Repo root: /content


In [ ]:
!pip install -r requirements.txt
!pip install gdown
!pip install arm_pytorch_utilities==0.4.3

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.5/508.5 kB 40.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 85.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 48.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.8/400.8 kB 33.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
!bash -c "cd pldm_envs/wall && bash presaved_datasets/download_all.sh"
!bash -c "cd pldm_envs/wall && mkdir -p presaved_datasets/rendered"
!bash -c "cd pldm_envs/wall && PYTHONPATH=../.. python render_images.py --input_path presaved_datasets/ds_size_634_no_images.npz --output_path presaved_datasets/rendered/ds_size_634.npz --config configs/good_quality_data.yaml"

Downloading...
From (original): https://drive.google.com/uc?id=1NwR-ui-akIgR2xcoJHYiHjOk9U5bYCa0
From (redirected): https://drive.google.com/uc?id=1NwR-ui-akIgR2xcoJHYiHjOk9U5bYCa0&confirm=t&uuid=c8bbbb97-f8ea-446a-8ee8-723d2e680047
To: /content/pldm_envs/wall/presaved_datasets/wall_data.tar.gz
100% 535M/535M [00:12<00:00, 43.7MB/s]
ds_size_1269_no_images.npz
ds_size_1500K_no_images.npz
ds_size_20312_no_images.npz
ds_size_325K_no_images.npz
ds_size_5078_no_images.npz
ds_size_634_no_images.npz
ds_size_81250_no_images.npz
good_quality_data_no_images.npz
len_17_no_images.npz
len_33_no_images.npz
len_65_no_images.npz
noise_mix_0.001_no_images.npz
noise_mix_0.01_no_images.npz
noise_mix_0.02_no_images.npz
noise_mix_0.04_no_images.npz
noise_mix_0.08_no_images.npz
noise_mix_0.16_no_images.npz
random_trajectories_no_images.npz
wc_rate_0.npz
100% 1/1 [00:00<00:00,  4.47it/s]


In [ ]:
!python PLDM_hieral/generate_wall_trials.py \
  --config PLDM_hieral/configs/tworooms_l1.yaml \
  --output_train PLDM_hieral/wall_trials_train.npz \
  --output_eval PLDM_hieral/wall_trials_eval.npz \
  --n_eval 100 \
  --seed 42


Saved train trials to PLDM_hieral/wall_trials_train.npz
Saved eval trials to PLDM_hieral/wall_trials_eval.npz


## Run L1 2M baseline (no L2)
Original-size L1-only model (no parameter matching).


In [ ]:
!python PLDM_hieral/generate_wall_trials.py \
  --config PLDM_hieral/configs/tworooms_l1.yaml \
  --output_train PLDM_hieral/wall_trials_train.npz \
  --output_eval PLDM_hieral/wall_trials_eval.npz \
  --n_eval 100 \
  --seed 42

!CUDA_VISIBLE_DEVICES=0 python PLDM_hieral/run_tworooms_compare.py \
  --config_l1 PLDM_hieral/configs/tworooms_l1.yaml \
  --mode l1 \
  --output_root {OUTPUT_ROOT} \
  --epochs 600

Saved train trials to PLDM_hieral/wall_trials_train.npz
Saved eval trials to PLDM_hieral/wall_trials_eval.npz
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
estimating mean stds: 100% 100/100 [00:01<00:00, 85.19it/s] 
starting training for 600 epochs
resuming from /content/drive/MyDrive/PLDM_2mvs6m/tworooms_l1/epoch=330_sample_step=191232.ckpt
resumed from epoch 330 step 2978
Epoch:   0% 0/271 [00:00<?, ?it/s]
Batch:   0% 0/9 [00:00<?, ?it/s]
Batch:  11% 1/9 [00:01<00:14,  1.79s/it]
Batch:  44% 4/9 [00:01<00:01,  2.63it/s]
Batch:  67% 6/9 [00:02<00:00,  4.18it/s]
Batch: 100% 9/9 [00:02<00:00,  4.06it/s]
Epoch:   0% 1/271 [00:02<10:20,  2.30s/it]
Batch:   0% 0/9 [00:00<?, ?it/

## Run L1 (2M) vs L2 comparison
Runs L1-only (`tworooms_l1.yaml`) then L2 (`tworooms_l2.yaml`) from scratch.


In [ ]:
!python PLDM_hieral/generate_wall_trials.py \
  --config PLDM_hieral/configs/tworooms_l1.yaml \
  --output_train PLDM_hieral/wall_trials_train.npz \
  --output_eval PLDM_hieral/wall_trials_eval.npz \
  --n_eval 100 \
  --seed 42

!CUDA_VISIBLE_DEVICES=0 python PLDM_hieral/run_tworooms_compare.py \
  --config_l1 PLDM_hieral/configs/tworooms_l1.yaml \
  --config_l2 PLDM_hieral/configs/tworooms_l2.yaml \
  --output_root {OUTPUT_ROOT} \
  --l2_from_scratch \
  --mode both \
  --epochs 600


Streaming output truncated to the last 5000 lines.
Batch:  44% 4/9 [00:00<00:00, 12.00it/s]
Batch:  67% 6/9 [00:00<00:00, 11.99it/s]
Batch: 100% 9/9 [00:00<00:00, 11.97it/s]
Epoch:   7% 41/601 [00:31<07:02,  1.33it/s]
Batch:   0% 0/9 [00:00<?, ?it/s]
Batch:  22% 2/9 [00:00<00:00, 11.96it/s]
Batch:  44% 4/9 [00:00<00:00, 12.00it/s]
Batch:  67% 6/9 [00:00<00:00, 11.98it/s]
Batch: 100% 9/9 [00:00<00:00, 11.92it/s]
Epoch:   7% 42/601 [00:32<07:01,  1.33it/s]
Batch:   0% 0/9 [00:00<?, ?it/s]
Batch:  22% 2/9 [00:00<00:00, 12.04it/s]
Batch:  44% 4/9 [00:00<00:00, 11.61it/s]
Batch:  67% 6/9 [00:00<00:00, 11.68it/s]
Batch: 100% 9/9 [00:00<00:00, 11.66it/s]
Epoch:   7% 43/601 [00:33<07:03,  1.32it/s]
Batch:   0% 0/9 [00:00<?, ?it/s]
Batch:  22% 2/9 [00:00<00:00, 12.10it/s]
Batch:  44% 4/9 [00:00<00:00, 12.07it/s]
Batch:  67% 6/9 [00:00<00:00, 12.05it/s]
Batch: 100% 9/9 [00:00<00:00, 12.01it/s]
Epoch:   7% 44/601 [00:33<07:01,  1.32it/s]
Batch:   0% 0/9 [00:00<?, ?it/s]
Batch:  22% 2/9 [00:00<00:

Outputs are saved to Google Drive under `/content/drive/MyDrive/PLDM_2mvs6m`.


In [ ]:
# Outputs are already in Drive.
!ls -la {OUTPUT_ROOT}


total 9
-rw------- 1 root root  299 Jan 10 15:11 tworooms_compare.csv
drwx------ 3 root root 4096 Jan 10 14:15 tworooms_l1
drwx------ 3 root root 4096 Jan 10 15:11 tworooms_l2
